<a href="https://colab.research.google.com/github/cardstud/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/assignment_regression_classification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`). 

Use a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.

- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Fit a ridge regression model with multiple features.
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [32]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [33]:
df.describe()

,BLOCK,LOT,EASE-MENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE
count,23040.000000,23040.000000,0.0,23039.000000,23039.000000,23039.000000,23039.000000,2.303900e+04,23005.000000,23040.000000,2.304000e+04
mean,4459.116493,353.948481,NaN,10791.186987,1.827423,0.286514,2.320891,3.508555e+03,1819.671724,1.624479,1.328133e+06
std,3713.269887,628.025255,NaN,1103.414210,10.090432,5.778224,12.070554,2.218610e+04,488.376571,0.803898,1.025394e+07
min,1.000000,1.000000,NaN,0.000000,0.000000,-148.000000,0.000000,0.000000e+00,0.000000,1.000000,0.000000e+00
25%,1341.000000,22.000000,NaN,10306.000000,0.000000,0.000000,1.000000,4.940000e+02,1920.000000,1.000000,0.000000e+00
50%,3546.000000,49.000000,NaN,11211.000000,1.000000,0.000000,1.000000,1.356000e+03,1940.000000,1.000000,4.550000e+05
75%,6673.750000,375.000000,NaN,11360.000000,2.000000,0.000000,2.000000,2.280000e+03,1965.000000,2.000000,8.750000e+05
max,16350.000000,9057.000000,NaN,11697.000000,750.000000,570.000000,755.000000,1.303935e+06,2019.000000,4.000000,8.500000e+08


In [42]:
df.isnull().sum()

BOROUGH                               0
NEIGHBORHOOD                          0
BUILDING_CLASS_CATEGORY               0
TAX_CLASS_AT_PRESENT                  1
BLOCK                                 0
LOT                                   0
EASE-MENT                         23040
BUILDING_CLASS_AT_PRESENT             1
ADDRESS                               0
APARTMENT_NUMBER                  17839
ZIP_CODE                              1
RESIDENTIAL_UNITS                     1
COMMERCIAL_UNITS                      1
TOTAL_UNITS                           1
LAND_SQUARE_FEET                     53
GROSS_SQUARE_FEET                     1
YEAR_BUILT                           35
TAX_CLASS_AT_TIME_OF_SALE             0
BUILDING_CLASS_AT_TIME_OF_SALE        0
SALE_PRICE                            0
SALE_DATE                             0
dtype: int64

In [0]:
df = df.drop(['EASE-MENT', 'APARTMENT_NUMBER', 'LAND_SQUARE_FEET', 'YEAR_BUILT'], axis=1)

In [45]:
df.isnull().sum()

BOROUGH                           0
NEIGHBORHOOD                      0
BUILDING_CLASS_CATEGORY           0
TAX_CLASS_AT_PRESENT              1
BLOCK                             0
LOT                               0
BUILDING_CLASS_AT_PRESENT         1
ADDRESS                           0
ZIP_CODE                          1
RESIDENTIAL_UNITS                 1
COMMERCIAL_UNITS                  1
TOTAL_UNITS                       1
GROSS_SQUARE_FEET                 1
TAX_CLASS_AT_TIME_OF_SALE         0
BUILDING_CLASS_AT_TIME_OF_SALE    0
SALE_PRICE                        0
SALE_DATE                         0
dtype: int64

### Do train/test split. 
- Use data from January — March 2019 to train. Use data from April 2019 to test.

In [46]:
df.sort_values(by=['SALE_DATE'], ascending=True)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,R4,"447 WEST 18TH STREET, PH12A",10011.0,1.0,0.0,1.0,1979.0,2,R4,0,01/01/2019
35,3,BOROUGH PARK,07 RENTALS - WALKUP APARTMENTS,2A,5639,30,C2,1043 50TH STREET,11219.0,6.0,0.0,6.0,6006.0,2,C2,0,01/01/2019
36,3,OTHER,22 STORE BUILDINGS,4,8001,1,K1,8023 FLATLANDS AVENUE,11236.0,0.0,3.0,3.0,3838.0,4,K1,0,01/01/2019
37,3,OTHER,22 STORE BUILDINGS,4,8001,4,K1,8015 FLATLANDS AVENUE,11236.0,0.0,3.0,3.0,2650.0,4,K1,0,01/01/2019
38,3,OTHER,22 STORE BUILDINGS,4,8001,6,K1,8009 FLATLANDS AVENUE,11236.0,0.0,3.0,3.0,2680.0,4,K1,0,01/01/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22930,2,OTHER,02 TWO FAMILY DWELLINGS,1,4841,57,B1,714 EAST 228TH STREET,10466.0,2.0,0.0,2.0,1920.0,1,B1,730000,04/30/2019
22931,2,OTHER,02 TWO FAMILY DWELLINGS,1,4874,55,B2,1026 EAST 228 STREET,10466.0,2.0,0.0,2.0,2416.0,1,B2,735000,04/30/2019
22932,3,OTHER,17 CONDO COOPS,2,6411,1002,R9,"153 BAY 26TH STREET, 1",11214.0,0.0,0.0,0.0,0.0,2,R9,200000,04/30/2019
22934,3,OTHER,44 CONDO PARKING,4,5871,1022,RG,"268 BAY RIDGE AVENUE, P7",11220.0,0.0,1.0,1.0,153.0,4,RG,626224,04/30/2019


In [0]:
train = df[df['SALE_DATE'] < '04/01/2019']

In [48]:
print(train.shape)
train.head()


(18167, 17)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,R4,"447 WEST 18TH STREET, PH12A",10011.0,1.0,0.0,1.0,1979.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,O5,144 WEST 37TH STREET,10018.0,0.0,6.0,6.0,15435.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,O5,40 WEST 38TH STREET,10018.0,0.0,7.0,7.0,11332.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,R4,"1 SHERIDAN SQUARE, 8C",10014.0,1.0,0.0,1.0,500.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,R1,"20 EAST 65TH STREET, B",10065.0,1.0,0.0,1.0,6406.0,2,R1,0,01/01/2019


In [0]:
test = df[df['SALE_DATE'] >= '04/01/2019']

In [50]:
print(test.shape)
test.head()

(4873, 17)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
18167,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,695,1562,R4,"555 WEST 23RD STREET, N11E",10011.0,1.0,0.0,1.0,670.0,2,R4,949000,04/01/2019
18168,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,767,1609,R4,"221 WEST 17TH STREET, 6A",10011.0,1.0,0.0,13.0,36835.0,2,R4,4683950,04/01/2019
18169,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,767,1612,R4,"221 WEST 17TH STREET, PH3",10011.0,1.0,0.0,13.0,36835.0,2,R4,5600000,04/01/2019
18170,1,OTHER,17 CONDO COOPS,2,721,1008,R9,"465 WEST 23RD STREET, RCU1",10011.0,0.0,0.0,0.0,0.0,2,R9,1100000,04/01/2019
18171,1,OTHER,21 OFFICE BUILDINGS,4,746,64,O8,340 WEST 23RD STREET,10011.0,3.0,1.0,4.0,5603.0,4,O8,0,04/01/2019


In [51]:
train.select_dtypes(exclude='number').describe().T

,count,unique,top,freq
BOROUGH,18167,5,4,5883
NEIGHBORHOOD,18167,11,OTHER,15058
BUILDING_CLASS_CATEGORY,18167,43,01 ONE FAMILY DWELLINGS,4094
TAX_CLASS_AT_PRESENT,18167,10,1,8911
BUILDING_CLASS_AT_PRESENT,18167,140,D4,2640
ADDRESS,18167,17926,N/A ROCKAWAY BOULEVARD,5
BUILDING_CLASS_AT_TIME_OF_SALE,18167,140,D4,2640
SALE_DATE,18167,90,01/24/2019,480


In [52]:
# Check relationship of Borough with SALE_PRICE
train.groupby('BOROUGH')['SALE_PRICE'].mean()

BOROUGH
1    3.200017e+06
2    5.736442e+05
3    8.495242e+05
4    7.335611e+05
5    3.947132e+05
Name: SALE_PRICE, dtype: float64

In [53]:
train.groupby('NEIGHBORHOOD')['SALE_PRICE'].mean()

NEIGHBORHOOD
ASTORIA                    2.134987e+06
BEDFORD STUYVESANT         9.473171e+05
BOROUGH PARK               4.562094e+05
FLUSHING-NORTH             6.626492e+05
FOREST HILLS               5.214604e+05
GRAMERCY                   8.226612e+05
OTHER                      1.197267e+06
UPPER EAST SIDE (59-79)    3.335407e+06
UPPER EAST SIDE (79-96)    1.737515e+06
UPPER WEST SIDE (59-79)    1.302832e+06
UPPER WEST SIDE (79-96)    1.365809e+06
Name: SALE_PRICE, dtype: float64

In [54]:
train.groupby('TAX_CLASS_AT_PRESENT')['SALE_PRICE'].mean()

TAX_CLASS_AT_PRESENT
1     4.905709e+05
1A    3.615866e+05
1B    7.556957e+05
1C    1.051295e+06
1D    3.582324e+05
2     1.345565e+06
2A    8.459622e+05
2B    1.785369e+06
2C    1.102988e+06
4     6.186266e+06
Name: SALE_PRICE, dtype: float64

In [55]:
train.groupby('SALE_DATE')['SALE_PRICE'].mean()

SALE_DATE
01/01/2019    1.153846e+04
01/02/2019    5.415812e+05
01/03/2019    9.637578e+05
01/04/2019    6.682856e+05
01/05/2019    5.736461e+04
                  ...     
03/27/2019    1.205941e+06
03/28/2019    1.010468e+06
03/29/2019    8.007240e+05
03/30/2019    7.491100e+04
03/31/2019    0.000000e+00
Name: SALE_PRICE, Length: 90, dtype: float64

In [56]:
train.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'BUILDING_CLASS_AT_PRESENT',
       'ADDRESS', 'ZIP_CODE', 'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS',
       'TOTAL_UNITS', 'GROSS_SQUARE_FEET', 'TAX_CLASS_AT_TIME_OF_SALE',
       'BUILDING_CLASS_AT_TIME_OF_SALE', 'SALE_PRICE', 'SALE_DATE'],
      dtype='object')

In [59]:
target = ['SALE_PRICE']
high_cardinality = ([['BUILDING_CLASS_CATEGORY','BLOCK', 'LOT',
                    'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 
                    'ZIP_CODE','RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS',
                    'TOTAL_UNITS', 'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 
                    'YEAR_BUILT', 'TAX_CLASS_AT_TIME_OF_SALE',
                    'BUILDING_CLASS_AT_TIME_OF_SALE']])
features = train.columns.drop([target] + high_cardinality)

X_train = train[features]
y_train = train[target]
X_test = teste[features]
y_test = test[target]

TypeError: ignored